# __Create a Text Classification Model__

- Tutorial Difficulty: ★☆☆☆☆
- 10 min read
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_ml/classification/text_classification.ipynb
- References: [(Kaggle) IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data), [ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators](https://arxiv.org/abs/2003.10555)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Classification</h4>
    <p>Classification is a type of <a href="https://en.wikipedia.org/wiki/Machine_learning">Machine Learning</a> that predicts which category(Category or Class) the target belongs to. For example, both binary classifications(used for classifying men or women) and multiple classifications(used to predict animal species such as dogs, cats, rabbits, etc.) are included in the classification tasks. <br></p>
</div>

[Natural Language Processing(NLP)](https://en.wikipedia.org/wiki/Natural_language_processing) is a branch of artificial intelligence that uses machine learning to process and interpret text-based data.

<div class="admonition tip">
    <h4 class="admonition-title">What is <a href="https://en.wikipedia.org/wiki/Natural_language_processing">Natural Language Processing(NLP)</a></h4>
    <p>Depending on the task, NLP can be divided into two categories: Natural Language Understanding(NLU) and Natural Language Generation(NLG). NLU is the process of converting a person's natural language into a value that a computer can understand. NLG, on the other hand, refers to the process of translating computer-readable values into natural language that humans can understand.</p>
</div>

Recent advancements in pre-training techniques, such as [BERT](<https://en.wikipedia.org/wiki/BERT_(language_model)>) and [GPT-3](https://en.wikipedia.org/wiki/GPT-3), allow for the development of a common language comprehension model prior to fine-tuning for specific NLP tasks, such as emotional analysis or question-and-answer.

<div class="admonition summary">
    <p>This means that you can use data more efficiently by minimizing <a href="https://en.wikipedia.org/wiki/Labeled_data">data labeling</a> operations for large datasets.</p>
</div>

ThanoSQL includes a variety of pre-trained AI models along with various functions that allow users to easily create their own text classification models even with limited data labeling. Users can use this to extract potentially useful insights from text data that would otherwise be difficult and apply them to a wide range of applications.

__The following are examples and applications of the ThanoSQL text classification model.__

- The ThanoSQL text classification model makes it easy to utilize text classification models to build a chatbot and analyze sentiment of text in a bulletin board. This will later enable the customer to connect with the appropriate customer service representative.

- The ThanoSQL text classification model allows news or post sharing services to categorize their published contents into groups. Additionally, it provides sentiment analysis of grouped items, enabling effective handling of issues that could arise unexpectedly from customer frustration response.

<div class="admonition note">
    <h4 class="admonition-title">In This Tutorial</h4>
    <p>👉 Create a model to classify the emotions of movie reviews using the IMDB Movie Reviews dataset from <a href="https://www.kaggle.com/">Kaggle</a>. This dataset consists of 50,000 movie review texts and each reviews are rated as either positive or negative. Based on the movie rating, a value less than 5 is expressed as negative and a value greater than 7 is expressed as positive. Each film has no more than 30 reviews.</p>
</div>

<div class="admonition warning">
    <h4 class="admonition-title">Tutorial Precautions</h4>
    <ul>
        <li>A text classification model can be used to predict one target value(Target, Category) from one text value.</li>
        <li>Both a column representing the text and a column representing the target value of the text must exist.</li>
        <li>The base model of the corresponding text classification model(<strong>ELECTRA</strong>) uses GPU. Depending on the size and the batch size of the model used, GPU memory may be insufficient. In this case, try using a smaller model or reducing the batch size of the model.</li>
    </ul>
</div>

## __0. Prepare Dataset and Model__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/paas/workspace/lab/), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY movie_review_train
OPTIONS (if_exists='replace') 
FROM 'thanosql-dataset/movie_review_data/movie_review_train.csv'

Success


In [4]:
%%thanosql
COPY movie_review_test 
OPTIONS (if_exists='replace') 
FROM 'thanosql-dataset/movie_review_data/movie_review_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"if_exists": determines how the function should handle the case where the table already exists, it can either raise an error, append to the existing table, or replace the existing table (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

### __Prepare the Model__

In [5]:
%%thanosql
GET THANOSQL MODEL electra
OPTIONS (
    model_name='tutorial_text_classification',
    overwrite=True
    )

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL MODEL</strong>" downloads the specified model to the workspace.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL MODEL</strong> clause.
        <ul>
            <li>"model_name": the model name to store a given model in the ThanoSQL workspace (str, optional)</li>
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1.Check Dataset__

To create a movie review sentiment classification model, we use the __movie_review_train__ table from the ThanoSQL workspace database. To check the table's contents, run the following query.

In [6]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data Table</h4>
   <p><strong>movie_review_train</strong> table contains the following information.</p>
   <ul>
      <li>review: movie review in text format</li>
      <li>sentiment: target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __2. Predict Using Pre-built Model__

To predict the results using the pre-built <strong>tutorial_text_classification</strong> model, run the query below.

In [7]:
%%thanosql
PREDICT USING tutorial_text_classification
OPTIONS (
    text_col='review'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predict_result
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,positive
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive


## __3. Build a Text Classification Model__

To create a text classification model with the name __my_movie_review_classifier__ using the <strong>movie_review_train</strong> table, run the following query.  
(Estimated duration of query execution: 3 min)

In [8]:
%%thanosql
BUILD MODEL my_movie_review_classifier
USING ElectraEn
OPTIONS (
    text_col='review',
    label_col='sentiment',
    max_epochs=1,
    batch_size=4,
    overwrite=True
    )
AS
SELECT *
FROM movie_review_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <strong>my_movie_review_classifier</strong>.</li>
        <li>"<strong>USING</strong>" specifies <strong>ElectraEn</strong> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create the model.
        <ul>
            <li> "text_col": the name of the column containing the text to be used for the training (str, default: 'text')</li> 
            <li> "label_col": the name of the column containing information about the target (str, default: 'label')</li> 
            <li>"max_epochs": number of times to train with the training dataset (int, optional, default: 3)</li>
            <li> "batch_size": the size of dataset bundle utilized in a single cycle of training (int, optional, default: 16)</li>
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

<div class="admonition tip">
    <p>In this example, we set "max_epochs" to 1 to train the model quickly. In general, larger number of "max_epochs" increases performance of the inference at the cost of the computation time.</p>
</div>

## __4. Predict Movie Review Sentiment__

To use the text classification model created in the previous step for prediction of __movie_review_test__, run the following query.

In [9]:
%%thanosql
PREDICT USING my_movie_review_classifier
OPTIONS (
    text_col='review',
    result_col='predict_result'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predict_result
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,negative
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" predicts the outcome using the <strong>my_movie_review_classifier</strong>.
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for prediction.
        <ul>
            <li>"text_col": the column containing the text to be used for prediction (str, default: 'text')</li>
            <li>"result_col": the column that contains the predicted results (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __5. In Conclusion__

In this tutorial, we created a text classification model using the IMDB Movie Reviews dataset. As this is a beginner-level tutorial, we focused on the process rather than accuracy. Text classification models can be improved in accuracy through fine tuning that is suitable for the user's needs. You can train the base model using your own data, or use a [Self-supervised Learning](https://en.wikipedia.org/wiki/Self-supervised_learning) model to vectorize and transform your data to create an automated machine learning(AutoML) for deployment. Create your own model and provide competitive services by combining various unstructured data(image, audio, video, etc.) and structured data with ThanoSQL.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries About Deploying a Model for Your Own Service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your service, please feel free to contact us below😊</p>
    <p>For inquiries regarding building a text classification model: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>